In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [ ]:
using_colab = True


In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [ ]:
image = cv2.imread('images/000001-img.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#img = cv2.imread('your file name')

height = image.shape[0]
width = image.shape[1]

# Cut the image in half
width_cutoff = width // 2
s1 = image[:, :width_cutoff]
s2 = image[:, width_cutoff:]

cv2.imwrite("part1.jpg", s1)
cv2.imwrite("part2.jpg", s2)

In [ ]:
original_img = cv2.imread('part1.jpg')
#original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
recon_img = cv2.imread('part2.jpg')
#recon_img = cv2.cvtColor(recon_img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(original_img)
plt.axis('on')
plt.show()

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
predictor.set_image(original_img)


In [ ]:
input_point = np.array([[620, 500]])
input_label = np.array([1])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(original_img)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)

In [ ]:
masks.shape  # (number_of_masks) x H x W

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(original_img)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()

In [ ]:
input_point = np.array([[1000, 500], [1250, 500], [700, 500]])
input_label = np.array([1, 1, 1])

mask_input = logits[np.argmax(scores), :, :]  # Choose the model's best mask

In [ ]:
masks, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
masks.shape
def masking(image,masks):
  h,w,bpp = np.shape(image)
  newimage = image.copy()

  for py in range(0,h):
      for px in range(0,w):
#can change the below logic of rgb according to requirements. In this
#white background is changed to #e8e8e8  corresponding to 232,232,232
#intensity, red color of the image is retained.
          if(masks[0][py][px] == False):
              newimage[py][px][0]=225
              newimage[py][px][1]=225
              newimage[py][px][2]=225



  newimage=cv2.cvtColor(newimage, cv2.COLOR_BGR2RGB)
  return newimage

In [ ]:
img1 = masking(original_img,masks)
img2 = masking(recon_img,masks)


cv2.imwrite('image15.jpg',img1)
cv2.imwrite('image16.jpg',img2)